Code used to find the optimal number of past values and number of neurons for the final neural network. 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from keras.layers import concatenate
from keras.models import Sequential
from keras.layers import Activation, Dense
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import math
import keras
from keras import backend as K 

In [2]:
"""Learning to supervised model"""
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [3]:
room = pd.read_csv('/home/ict4bd/Residential_oslo/simulation_data/bath.csv', parse_dates=[0,], index_col="Date_Time")
#if you want to study univariate MLP, remove the comment on the following line, the only column remaining will be 't_in'
#room = room.drop(['t_out', 'power_h1', 'power_c1', 'solar_rad'], axis=1)
number_features = len(room.columns)

In [4]:
room.columns

Index(['t_in'], dtype='object')

In [5]:
values = room.values #change this according to the room in study
values = values.astype('float32') # ensure all data is float
number_future_samples = 20 #up to five hours

In [14]:
#OPTIMIZATION LAG
#find the best lag order considering from 1 to 15 past values

MAE_Past_Values_List = [] #store the value of average prediction error for each lag order

for number_past_values in range (1,10):

  #Preparation of the dataset
  reframed = series_to_supervised(values, number_past_values, number_future_samples)
  input_past_values = reframed.iloc[:,0:number_past_values * number_features]
  output_future_values = reframed.iloc[:,number_past_values * number_features:]
  output_future_values = reframed.filter(regex=r'(var1\(t\+|var1\(t\))')
  prepared_dataset = pd.concat([input_past_values, output_future_values], axis = 1 )

  #Splitting dataset into training tand test set
  to_be_scaled_values = prepared_dataset.values
  scaler = MinMaxScaler(feature_range=(0, 1))
  scaled_values = scaler.fit_transform(to_be_scaled_values)
  n_train_hours = 3 * 365 * 24 * 4  #number of quaters of hour in 3 years time 

  train = scaled_values[:n_train_hours, :]
  test = scaled_values[n_train_hours:, :]
  train_X, train_y = train[:, :-(number_future_samples)], train[:, -(number_future_samples):]
  test_X, test_y = test[:, :-(number_future_samples)], test[:, -(number_future_samples):]

  #Definition of the mode
  model = Sequential()
  model.add(Dense(30, activation='sigmoid', input_dim = train_X.shape[1],kernel_initializer='random_normal')) 
  model.add(Dense(number_future_samples))
  optimizer = keras.optimizers.Adam(lr=0.01)
  model.compile(optimizer=optimizer, loss='MAE')

  #Model Fitting doing the average of three fitting
  yhat_list = np.zeros((test_X.shape[0],number_future_samples))
  j = 0
  for i in range (1,4):
    history = model.fit(train_X, train_y, epochs = 100, batch_size=72, validation_data=(test_X, test_y), verbose=2, shuffle=False)
    #Prediction
    yhat = model.predict(test_X)
    yhat_list = yhat_list + yhat
    j = j + 1
  yhat = yhat_list/j
  
  #Invert scaling of the test vector 
  inv_test_y = concatenate((test_X,test_y),axis = 1)
  inv_test_y = scaler.inverse_transform(inv_test_y)
  inv_test_y = inv_test_y[:,-number_future_samples:]

 #Invert scaling predicted y_hat
  inv_yhat = concatenate((test_X,yhat),axis = 1)
  inv_yhat = np.array(inv_yhat)
  inv_yhat = scaler.inverse_transform(inv_yhat)
  inv_yhat = inv_yhat[:,-number_future_samples:]

  MAE_list = [] #vector storing the MAE of the future prediction (from the next step to 20 step ahead)
  for i in range (inv_yhat.shape[1]):
    MAE = mean_absolute_error(inv_test_y[:,i],inv_yhat[:,i])
    MAE_list.append(MAE)

  MAE_Past_Values_List.append(MAE_list)
  MAE_list = np.array(MAE_list)

  print('%d past value - MAE %.4f' %(number_past_values,MAE_list.mean())) #Append the average prediction error 

  del model
  K.clear_session()

Epoch 1/5
1460/1460 - 2s - loss: 0.0717 - val_loss: 0.0496
Epoch 2/5
1460/1460 - 2s - loss: 0.0395 - val_loss: 0.0494
Epoch 3/5
1460/1460 - 2s - loss: 0.0377 - val_loss: 0.0500
Epoch 4/5
1460/1460 - 2s - loss: 0.0360 - val_loss: 0.0464
Epoch 5/5
1460/1460 - 2s - loss: 0.0348 - val_loss: 0.0434
1 past value - MAE 0.5308
Epoch 1/5
1460/1460 - 2s - loss: 0.0574 - val_loss: 0.0595
Epoch 2/5
1460/1460 - 2s - loss: 0.0413 - val_loss: 0.0629
Epoch 3/5
1460/1460 - 1s - loss: 0.0390 - val_loss: 0.0566
Epoch 4/5
1460/1460 - 2s - loss: 0.0371 - val_loss: 0.0544
Epoch 5/5
1460/1460 - 2s - loss: 0.0359 - val_loss: 0.0498
2 past value - MAE 0.6092
Epoch 1/5
1460/1460 - 2s - loss: 0.0572 - val_loss: 0.0690
Epoch 2/5
1460/1460 - 2s - loss: 0.0429 - val_loss: 0.0652
Epoch 3/5
1460/1460 - 2s - loss: 0.0396 - val_loss: 0.0564
Epoch 4/5
1460/1460 - 2s - loss: 0.0372 - val_loss: 0.0570
Epoch 5/5
1460/1460 - 2s - loss: 0.0362 - val_loss: 0.0532
3 past value - MAE 0.6500
Epoch 1/5
1460/1460 - 2s - loss: 0.05

In [13]:
MAE_Past_Values_List = np.array(MAE_Past_Values_List)

optimal_lag_value = np.argmin(MAE_Past_Values_List) + 1
optimal_lag_value


1

In [16]:
#OPTIMIZATION NUMBER NEURONS

neuron_number_list = np.arange(1,15,1)

for number_past_values in [optimal_lag_value]:

  #Preparation of the dataset
  reframed = series_to_supervised(values, number_past_values, number_future_samples)
  input_past_values = reframed.iloc[:,0:number_past_values * number_features]
  output_future_values = reframed.iloc[:,number_past_values * number_features:]
  output_future_values = reframed.filter(regex=r'(var1\(t\+|var1\(t\))')
  prepared_dataset = pd.concat([input_past_values, output_future_values], axis = 1 )

  #Splitting dataset into training tand test set
  to_be_scaled_values = prepared_dataset.values
  scaler = MinMaxScaler(feature_range=(0, 1))
  scaled_values = scaler.fit_transform(to_be_scaled_values)
  n_train_hours = 3 * 365 * 24 * 4  #number of quaters of hour in 3 years time 

  train = scaled_values[:n_train_hours, :]
  test = scaled_values[n_train_hours:, :]
  train_X, train_y = train[:, :-(number_future_samples)], train[:, -(number_future_samples):]
  test_X, test_y = test[:, :-(number_future_samples)], test[:, -(number_future_samples):]

  MAE_neuron_number = []

  #Grid Search Number Neurons:
  for neuron_number in neuron_number_list:
    model = Sequential()
    model.add(Dense(neuron_number, activation='sigmoid', input_dim = train_X.shape[1])) 
    model.add(Dense(number_future_samples))
    optimizer = keras.optimizers.Adam(lr=0.01)
    model.compile(optimizer=optimizer, loss='mae')

    #Model Fitting
    history = model.fit(train_X, train_y, epochs = 200, batch_size=72, validation_data=(test_X, test_y), verbose=0, shuffle=False)
    #Prediction
    yhat = model.predict(test_X)

    #Invert scaling of the test vector 
    inv_test_y = concatenate((test_X,test_y),axis = 1)
    inv_test_y = scaler.inverse_transform(inv_test_y)
    inv_test_y = inv_test_y[:,-number_future_samples:]

  #Invert scaling predicted y_hat
    inv_yhat = concatenate((test_X,yhat),axis = 1)
    inv_yhat = np.array(inv_yhat)
    inv_yhat = scaler.inverse_transform(inv_yhat)
    inv_yhat = inv_yhat[:,-number_future_samples:]

    MAE_future_step_neuron = [] #store the MAE at everychange of number of neurons
    for i in range (inv_yhat.shape[1]):
      MAE = mean_absolute_error(inv_test_y[:,i],inv_yhat[:,i])
      MAE_future_step_neuron.append(MAE)
    MAE_neuron_number.append(MAE_future_step_neuron)

    print('Training past value %d with %d neurons over.' %(number_past_values, neuron_number))
  
  MAE_neuron_number = np.array(MAE_neuron_number)
  MAE_variation_neurons = []
  for i in range (MAE_neuron_number.shape[0]):
    MAE_variation_neurons.append(MAE_neuron_number[i,:].mean())
  optimal_neuron_number_index = np.argmin(MAE_variation_neurons)
  optimal_neuron_number = neuron_number_list[optimal_neuron_number_index]
  
  print ('Optimal number of neurons for past value %d = %d' %(number_past_values, optimal_neuron_number))

Training past value 1 with 1 neurons over.
Training past value 1 with 2 neurons over.
Training past value 1 with 3 neurons over.
Training past value 1 with 4 neurons over.
Training past value 1 with 5 neurons over.
Training past value 1 with 6 neurons over.
Training past value 1 with 7 neurons over.
Training past value 1 with 8 neurons over.
Training past value 1 with 9 neurons over.
Training past value 1 with 10 neurons over.
Training past value 1 with 11 neurons over.
Training past value 1 with 12 neurons over.
Training past value 1 with 13 neurons over.
Training past value 1 with 14 neurons over.
Optimal number of neurons for past value 1 = 6
